In [67]:
import os, glob
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import numpy as np
from datetime import date
import unicodedata


# Ruta base
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G")

HEADERS = [
    "eNodeB Name","CellName","activePlmnList_mcc","additionalPlmnList_mcc",
    "administrativeState","cellBarred","cellId","cellSubscriptionCapacity",
    "channelSelectionSetSize","dlChannelBandwidth","earfcndl","earfcnul",
    "freqBand","noOfPucchCqiUsers","noOfPucchSrUsers","operationalState",
    "physicalLayerCellIdGroup","physicalLayerSubCellId","sectorCarrierRef",
    "tac","timeOfLastModification","ulChannelBandwidth",
    "eNBId","eNodeB Name Unique","LAT","LON","PCI","AT&T_Site_Name",
    "MOCN Activo por Celda","Al menos una celda de MOCN encendida","MME TEF",
    "Gestor"  # <- nueva columna al final
]




In [68]:

import re

def appendfiles(filenamepattern: str) -> str:
    """
    Integra todos los TXT que matchean pattern + '_*.txt' en un solo archivo,
    agregando como ÚLTIMA COLUMNA el 'Gestor' derivado del sufijo numérico del archivo.
    Devuelve el nombre del archivo integrado (sin ruta).
    """
    searchpattern = str(BASE_DIR / f"{filenamepattern}_*.txt")
    filestoread = glob.glob(searchpattern)

    outputfile_name = f"Integrated_{filenamepattern}_files.txt"
    output_path = BASE_DIR / outputfile_name

    print("Buscando:", searchpattern)
    print("Archivos:", filestoread)

    with open(output_path, "w", encoding="utf-8") as outputfile:
        for name in filestoread:
            # Extrae número después del guion bajo: *_NN.txt
            m = re.search(r"_(\d+)\.txt$", os.path.basename(name), flags=re.IGNORECASE)
            gestor = m.group(1) if m else ""

            with open(name, "r", encoding="utf-8") as f:
                for line in f:
                    # evita líneas vacías puras
                    if not line.strip():
                        continue
                    # agrega el Gestor como última columna
                    line = line.rstrip("\n")
                    outputfile.write(f"{line}\t{gestor}\n")
            print("Agregado:", name)

    print("Integrado =>", outputfile_name)
    return outputfile_name



def cleanfile(filename: str, ignorelines=None) -> str:
    """
    Elimina líneas que contengan cualquiera de los patrones indicados.
    Devuelve el nombre del archivo limpio (sin ruta).
    """
    if ignorelines is None:
        ignorelines = ["SubNetwork,", "instance(s)", "NodeId"]

    inputfile = BASE_DIR / filename
    cleanfile_name = f"Clean_{filename}"
    cleanfile_path = BASE_DIR / cleanfile_name

    with open(inputfile, 'r', encoding="utf-8") as f_in:
        lines = f_in.readlines()

    kept = []
    for line in lines:
        if any(p in line for p in ignorelines):
            continue
        kept.append(line)

    with open(cleanfile_path, 'w', encoding="utf-8") as f_out:
        f_out.writelines(kept)

    print(f"Limpieza OK -> {cleanfile_name} ({len(kept)} líneas)")
    return cleanfile_name


def convert_to_excel(cleanfile_name: str) -> str:
    """
    Lee TXT tab-delimited sin encabezados y guarda a Excel.
    Devuelve el nombre del archivo Excel (sin ruta).
    """
    cleanfile_path = BASE_DIR / cleanfile_name
    out_xlsx = f"Converted_{cleanfile_name}.xlsx"
    out_path = BASE_DIR / out_xlsx

    df = pd.read_csv(cleanfile_path, delimiter='\t', header=None)
    df.to_excel(out_path, index=False, header=None)
    print(f"Convertido a Excel -> {out_xlsx}  (shape={df.shape})")
    return out_xlsx



In [69]:

# EUtranCellFDD
eu_txt = appendfiles('EUtranCellFDD')
eu_clean = cleanfile(eu_txt)
eu_xlsx = convert_to_excel(eu_clean)

# ENodeBFunction
nb_txt = appendfiles('ENodeBFunction')
nb_clean = cleanfile(nb_txt)
nb_xlsx = convert_to_excel(nb_clean)

# nodeid
nd_txt = appendfiles('nodeid')
nd_clean = cleanfile(nd_txt)
nd_xlsx = convert_to_excel(nd_clean)

# MME
mme_txt = appendfiles('MME')
mme_clean = cleanfile(mme_txt)
mme_xlsx = convert_to_excel(mme_clean)
#NblotCell
nbiot_xlsx = appendfiles('NbIotCell')
nbiot_clean = cleanfile(nbiot_xlsx)
nbiot_xlsx = convert_to_excel(nbiot_clean)



Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\EUtranCellFDD_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\EUtranCellFDD_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\EUtranCellFDD_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\EUtranCellFDD_9.txt
Integrado => Integrated_EUtranCellFDD_files.txt
Limpieza OK -> Clean_Integrated_EUtranCellFDD_files.txt (51814 líneas)


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_20748\2035501895.py:76: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cleanfile_path, delimiter='\t', header=None)


Convertido a Excel -> Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx  (shape=(51814, 24))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\ENodeBFunction_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\ENodeBFunction_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\ENodeBFunction_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\ENodeBFunction_9.txt
Integrado => Integrated_ENodeBFunction_files.txt
Limpieza OK -> Clean_Integrated_ENodeBFunction_files.txt (6937 líneas)
Convertido a Excel -> Converted_Clean_Integrated_ENodeBFunction_files.txt.xlsx  (shape=(6937, 4))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\nodeid_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\nodeid_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\L

In [70]:

# Archivo base desde la conversión de EUtranCellFDD
wb = load_workbook(BASE_DIR / eu_xlsx)  # ej. Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx
ws = wb.active

ultima_fila = ws.max_row

# 1) Eliminar la columna B (índice 2). Todo se recorre a la izquierda automáticamente.
ws.delete_cols(2)



wb.save(BASE_DIR / "Modified_workfile.xlsx")
print("Reacomodo OK -> Modified_workfile.xlsx")



Reacomodo OK -> Modified_workfile.xlsx


In [71]:
# === Tras leer el reacomodado SIN headers ===
df_base = pd.read_excel(BASE_DIR / "Modified_workfile.xlsx", header=None, dtype=str)

# 1) Identifica hasta dónde llega el bloque base del TXT (incluye 'ulChannelBandwidth')
BASE_LAST = "ulChannelBandwidth"
base_len = HEADERS.index(BASE_LAST) + 1   # columnas A..(ulChannelBandwidth)

n_cols = df_base.shape[1]

# 2) Si hay exactamente una columna extra después del bloque base,
#    interprétala como 'Gestor' (EUtranCellFDD con Gestor pegado al final del TXT)
gestor_series = pd.Series(pd.NA, index=df_base.index)
if n_cols >= base_len + 1:
    # La columna inmediatamente después del bloque base es el Gestor del EUtranCellFDD
    gestor_series = df_base.iloc[:, base_len]
    # quita esa columna física para no “correr” el resto
    df_base.drop(df_base.columns[base_len], axis=1, inplace=True)
    n_cols -= 1

# 3) Ahora asigna nombres al dataframe SIN todavía incluir 'Gestor'
#    (para no desalinear nada). Rellena columnas faltantes si hiciera falta.
expected_without_gestor = len(HEADERS) - 1  # todo excepto 'Gestor'
if n_cols < expected_without_gestor:
    for i in range(expected_without_gestor - n_cols):
        df_base[f"__tmp_empty_{i}"] = pd.NA
    n_cols = expected_without_gestor

# Corta/exacta y nombra SIN 'Gestor'
df_base = df_base.iloc[:, :expected_without_gestor]
df_base.columns = HEADERS[:-1]  # todos menos 'Gestor'

# 4) Inserta 'Gestor' con lo extraído (o NA si no hubo)
df_base["Gestor"] = gestor_series

# 5) Reordena a HEADERS finales por si acaso
df_base = df_base.reindex(columns=HEADERS)

# (Opcional) guarda control
df_base.to_excel(BASE_DIR / "Modified_with_headers.xlsx", index=False)
print("EUtranCellFDD: 'Gestor' acoplado correctamente tras 'ulChannelBandwidth'.")


EUtranCellFDD: 'Gestor' acoplado correctamente tras 'ulChannelBandwidth'.


In [73]:
df_base = df_base.loc[:, [c for c in df_base.columns if c in HEADERS]]
for c in HEADERS:
    if c not in df_base.columns:
        df_base[c] = pd.NA
df_base = df_base[HEADERS]

# --- 2) Cargar NbIot sin headers (A..N => 14 cols) ---
df_nbiot = pd.read_excel(BASE_DIR / nbiot_xlsx, header=None, dtype=str)
if df_nbiot.shape[1] < 15:
    raise ValueError("NbIotCell debe tener al menos 15 columnas (incluyendo 'Gestor' como última).")
df_nbiot = df_nbiot.iloc[:, :15]  # A.O

# --- 3) Construir bloque NbIot alineado por POSICIÓN ---
# A->A, B->AF, C->B, D->C, E->E, F->F, G->G, H->K, I->L, J->P, K->R, L->D, M->S, N->T, O->Gestor
df_nb = pd.DataFrame(pd.NA, index=df_nbiot.index, columns=HEADERS)
df_nb["eNodeB Name"]                 = df_nbiot.iloc[:, 0]   # A -> A
df_nb["CellName"]                    = df_nbiot.iloc[:, 2]   # C -> B
df_nb["activePlmnList_mcc"]          = df_nbiot.iloc[:, 3]   # D -> C
df_nb["administrativeState"]         = df_nbiot.iloc[:, 4]   # E -> E
df_nb["cellBarred"]                  = df_nbiot.iloc[:, 5]   # F -> F
df_nb["cellId"]                      = df_nbiot.iloc[:, 6]   # G -> G
df_nb["earfcndl"]                    = df_nbiot.iloc[:, 7]   # H -> K
df_nb["earfcnul"]                    = df_nbiot.iloc[:, 8]   # I -> L
df_nb["operationalState"]            = df_nbiot.iloc[:, 9]   # J -> P
df_nb["physicalLayerCellIdGroup"]    = df_nbiot.iloc[:,10]   # K -> R
df_nb["additionalPlmnList_mcc"]      = df_nbiot.iloc[:,11]   # L -> D
df_nb["sectorCarrierRef"]            = df_nbiot.iloc[:,12]   # M -> S
df_nb["tac"]                         = df_nbiot.iloc[:,13]   # N -> T
df_nb["Gestor"]                      = df_nbiot.iloc[:,14]   # O -> Gestor (nueva)

# --- 4) Pegar debajo y guardar ---
df_out = pd.concat([df_base, df_nb], ignore_index=True)
df_out.to_excel(BASE_DIR / "Modified_with_headers.xlsx", index=False)
print("OK: NbIotCell agregado debajo, con B->AF y el resto según mapeo por posición.")

OK: NbIotCell agregado debajo, con B->AF y el resto según mapeo por posición.


In [74]:




# --- eNBId desde ENodeBFunction ---
df_nodeb = pd.read_excel(BASE_DIR / nb_xlsx, header=None, usecols=[0, 1, 2])
# Lee el archivo Excel convertido de ENodeBFunction (ruta 'nb_xlsx'),
# sin encabezado (header=None), y solo las 3 primeras columnas (0,1,2).

df_nodeb.columns = ["NodeId", "ENodeBFunctionId", "eNBIdnew"]
# Asigna nombres a las 3 columnas: NodeId (clave), ENodeBFunctionId (solo informativo),
# y eNBIdnew (el valor que queremos traer por JOIN).

df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str)
# Asegura que la columna clave en df_out sea string (evita mismatches de tipo).

df_nodeb["NodeId"] = df_nodeb["NodeId"].astype(str)
# Asegura que la clave en el catálogo (NodeId) también sea string.

df_nodeb = df_nodeb.drop_duplicates(subset=["NodeId"], keep="first")
# Si hay filas duplicadas por NodeId en el catálogo, conserva la primera
# para evitar que el merge genere duplicados.

df_tmp = df_out.merge(df_nodeb[["NodeId", "eNBIdnew"]],
                       left_on="eNodeB Name", right_on="NodeId", how="left")
# LEFT JOIN: por cada fila de df_out, busca en df_nodeb la fila con el mismo NodeId.
# - Clave izquierda: eNodeB Name (df_out)
# - Clave derecha: NodeId (df_nodeb)
# - how="left": conserva todas las filas de df_out aunque no haya match.

df_out["eNBId"] = df_tmp["eNBIdnew"]
# Copia (asigna) a df_out la columna eNBId con el valor traído (eNBIdnew).
# Nota: si no hubo match, quedará NaN.

# ============ Ordenar por eNodeB Name (A-Z) ============
df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str).str.strip()
df_out = df_out.sort_values(by="eNodeB Name", ascending=True).reset_index(drop=True)

# ============ eNodeB Name Unique (solo cuando cambia) ============
_name = df_out["eNodeB Name"].astype(str).fillna("").str.strip()
is_new = _name.ne(_name.shift())
df_out["eNodeB Name Unique"] = np.where(is_new & _name.ne(""), df_out["eNodeB Name"], "")


# --- LAT/LON/AT&T_Site_Name desde All_Ericsson_4G_{YYYYMM} (mes anterior) ---
# YYYYMM del mes anterior
# 1) YYYYMM del mes anterior
# 1) YYYYMM del mes anterior
today = date.today()
prev_year  = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm = f"{prev_year}{prev_month:02d}"

# 2) Fuente principal: All_Ericsson
ae_path = BASE_DIR / f"All_Ericsson_4G_{yyyymm}.xlsx"
ae_df = pd.read_excel(ae_path, usecols=["eNodeB Name", "LAT", "LON", "AT&T_Site_Name"])
ae_df["eNodeB Name"] = ae_df["eNodeB Name"].astype(str).str.strip()
ae_df = ae_df.drop_duplicates(subset=["eNodeB Name"], keep="first")

df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str).str.strip()

merged = df_out.merge(ae_df, on="eNodeB Name", how="left", suffixes=("", "_ae"))
def _is_blank(s):
    # NaN o string vacío (incluye espacios)
    return s.isna() | s.astype(str).str.strip().eq("")

for col in ["LAT", "LON", "AT&T_Site_Name"]:
    m = _is_blank(merged[col])
    merged.loc[m, col] = merged.loc[m, col + "_ae"]
    merged.drop(columns=[col + "_ae"], inplace=True)

# (Opcional) armoniza dtypes sin downcasting silencioso
merged = merged.infer_objects(copy=False)
def coalesce_to_single_column(df, variants, target):
    """
    Une varias columnas posibles (variants) en una sola columna 'target',
    tomando el primer no-nulo por fila y eliminando las columnas sobrantes.
    """
    present = [c for c in variants if c in df.columns]
    if not present:
        return df
    if target in present:
        cols = [target] + [c for c in present if c != target]
    else:
        cols = present
    merged_series = df[cols].bfill(axis=1).iloc[:, 0]
    df[target] = merged_series
    to_drop = [c for c in present if c != target]
    df.drop(columns=to_drop, inplace=True, errors="ignore")
    return df

faltan = (
    _is_blank(merged["LAT"]) |
    _is_blank(merged["LON"]) |
    _is_blank(merged["AT&T_Site_Name"])
)

if not faltan.any():
    # All_Ericsson llenó todo → no usar EPT
    df_out = merged
    print("All_Ericsson cubrió 100% (LAT/LON/AT&T_Site_Name). Se omite fallback EPT.")
else:
    print(f"Quedan {int(faltan.sum())} filas con faltantes. Se aplica fallback EPT…")

    # 3) Fallback EPT (solo completa donde aún falte)
    ept_matches = glob.glob(str(BASE_DIR / "EPT_ATT_UMTS_LTE_*.xlsx"))
    if ept_matches:
        ept_file = ept_matches[0]
        ept_sheets = ["EPT_3G_LTE_OUTDOOR", "PLAN_OUTDOOR", "EPT_3G_LTE_INDOOR", "PLAN_INDOOR", "Eventos_Especiales"]

        frames = []
        for sh in ept_sheets:
            try:
                tmp = pd.read_excel(ept_file, sheet_name=sh, engine="openpyxl")
                frames.append(tmp)
            except Exception:
                pass

        if frames:
            ept_df = pd.concat(frames, ignore_index=True)

            # --- Unificar columnas variantes ---
            ept_df = coalesce_to_single_column(ept_df, ["Cell Name", "CellName", "ATT_CELL_ID_Name"], "Cell Name")
            ept_df = coalesce_to_single_column(ept_df, ["Latitud"], "LAT")
            ept_df = coalesce_to_single_column(ept_df, ["Longitud"], "LON")
            ept_df = coalesce_to_single_column(ept_df, ["AT&T_Site_Name"], "AT&T_Site_Name")

           # Forzar LAT/LON a numérico (evita FutureWarning)
            for c in ["LAT", "LON"]:
                if c in ept_df.columns:
                    ept_df[c] = (
                        ept_df[c]
                        .astype(str)
                        .str.strip("[]")
                        .str.replace(",", "", regex=False)
                    )
                    ept_df[c] = pd.to_numeric(ept_df[c], errors="coerce")

            # Limpiar espacios en SiteName y Cell Name
            for c in ["Cell Name", "AT&T_Site_Name"]:
                if c in ept_df.columns:
                    ept_df[c] = ept_df[c].astype(str).str.strip()

            ept_df = ept_df.drop_duplicates(subset=["Cell Name"], keep="first")

            # --- asegurar llave en tu base ---
            created_temp_key = False
            if "Cell Name" not in merged.columns:
                if "CellName" in merged.columns:
                    merged["Cell Name"] = merged["CellName"]
                    created_temp_key = True
                else:
                    raise KeyError("No se encontró 'Cell Name' ni 'CellName' en el DataFrame base para hacer el join con EPT.")

            merged["Cell Name"] = merged["Cell Name"].astype(str).str.strip()

            # Lookup desde EPT
            ept_lookup = ept_df[["Cell Name", "LAT", "LON", "AT&T_Site_Name"]].rename(columns={
                "LAT": "LAT_ept",
                "LON": "LON_ept",
                "AT&T_Site_Name": "AT&T_Site_Name_ept"
            })

            # Merge
            merged = merged.merge(ept_lookup, on="Cell Name", how="left")

            # Completar SOLO faltantes
            need_lat  = _is_blank(merged["LAT"]) if "LAT" in merged.columns else pd.Series(False, index=merged.index)
            need_lon  = _is_blank(merged["LON"]) if "LON" in merged.columns else pd.Series(False, index=merged.index)
            need_site = _is_blank(merged["AT&T_Site_Name"]) if "AT&T_Site_Name" in merged.columns else pd.Series(False, index=merged.index)

            if "LAT_ept" in merged: merged.loc[need_lat,  "LAT"] = merged.loc[need_lat,  "LAT_ept"]
            if "LON_ept" in merged: merged.loc[need_lon,  "LON"] = merged.loc[need_lon,  "LON_ept"]
            if "AT&T_Site_Name_ept" in merged: merged.loc[need_site, "AT&T_Site_Name"] = merged.loc[need_site, "AT&T_Site_Name_ept"]

            # Limpieza
            merged.drop(columns=[c for c in ["LAT_ept", "LON_ept", "AT&T_Site_Name_ept"] if c in merged.columns], inplace=True)
            if created_temp_key:
                merged.drop(columns=["Cell Name"], inplace=True)

            print("Fallback EPT aplicado por 'Cell Name':", ept_file)
        else:
            print("No se pudieron leer hojas del EPT; se omite fallback.")
    else:
        print("No se encontró archivo EPT_ATT_UMTS_LTE_*.xlsx; se omite fallback.")

# 4) Guardar para que tu celda de formateo final (headers/estilos) tome este resultado
final_path = BASE_DIR / "Datos_Modified.xlsx"
merged.to_excel(final_path, index=False)
print("Guardado con All_Ericsson + fallback EPT ->", final_path)

# --- Calcular PCI = IF(R blank, Q, Q*3 + R) y guardar ---
q_col = "physicalLayerCellIdGroup"
r_col = "physicalLayerSubCellId"

merged[q_col] = pd.to_numeric(merged[q_col], errors="coerce")
merged[r_col] = pd.to_numeric(merged[r_col].astype(str).str.strip().replace({"": None}), errors="coerce")

merged["PCI"] = pd.Series(
    np.where(merged[r_col].isna(),
             merged[q_col],
             merged[q_col]*3 + merged[r_col]),
    index=merged.index,
    dtype="Int64"
)

# --- Marcar "MOCN Activo por Celda" (match EXACTO) y guardar ---
# Cadena exacta contra la que quieres comparar
pattern = "[{mncLength=3, mcc=334, mnc=90}, {mncLength=2, mcc=334, mnc=3}, {mncLength=2, mcc=1, mnc=1}, {mncLength=2, mcc=1, mnc=1}, {mncLength=2, mcc=1, mnc=1}]"

# Crear/actualizar la columna "MOCN Activo por Celda"
merged["MOCN Activo por Celda"] = np.where(
    merged["additionalPlmnList_mcc"].astype(str).str.strip() == pattern,
    "Si",
    "No"
)
# --- "Al menos una celda de MOCN encendida" basado en match eNodeB Name ∈ {AT&T_Site_Name con MOCN=SI} ---
# === "Al menos una celda de MOCN encendida" basado en eNodeB Name con MOCN=Si ===
truthy = {"si", "sí", "yes", "true", "1"}

# Normalizar columnas necesarias
enb = merged["eNodeB Name"].astype(str).str.strip()
mocn = merged["MOCN Activo por Celda"].astype(str).str.strip().str.lower()

# 1) Conjunto de eNodeB Name que tienen MOCN = "Si" (o equivalentes en truthy)
enbs_con_mocn = set(enb[mocn.isin(truthy)])

# 2) Marcar en cada fila si su eNodeB Name está en ese conjunto
merged["Al menos una celda de MOCN encendida"] = np.where(
    enb.isin(enbs_con_mocn),
    "Si",
    "No"
)
# --- eNBId desde MME ---
df_MME = pd.read_excel(BASE_DIR / mme_xlsx, header=None, usecols=[0,1,2], dtype=str)
df_MME.columns = ["NodeId", "eNodeBFunction", "TermPointToMmeId"]
df_MME["NodeId"] = df_MME["NodeId"].astype(str).str.strip()

# Filtrar solo NodeId de longitud 7
df_MME_7 = df_MME[df_MME["TermPointToMmeId"].str.len() == 7].copy()

# Conteo tipo COUNTIF: cuántas veces aparece cada NodeId
mme_counts = df_MME_7["NodeId"].value_counts()

# Normalizar clave y mapear conteo a tu base por eNodeB Name
df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str).str.strip()
merged["MME TEF"] = (
    df_out["eNodeB Name"].map(mme_counts).fillna(0).astype("Int64")
)



df_out = merged
# Actualiza df_out con el DataFrame enriquecido.

# --- Guardar preliminar (lo tomará la [7] para formateo final) ---
final_path = BASE_DIR / "Datos_Modified.xlsx"
# Define la ruta del Excel preliminar (sin estilos).

df_out.to_excel(final_path, index=False)
# Escribe el Excel con todas las columnas (aquí todavía sin formato openpyxl).

print("Guardado enriquecido ->", final_path, "shape=", df_out.shape)
# Log: confirma guardado y muestra dimensiones finales.





All_Ericsson cubrió 100% (LAT/LON/AT&T_Site_Name). Se omite fallback EPT.
Guardado con All_Ericsson + fallback EPT -> C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\Datos_Modified.xlsx
Guardado enriquecido -> C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\Datos_Modified.xlsx shape= (59331, 32)


In [75]:


final_excel = BASE_DIR / "Datos_Modified.xlsx"
tmp_excel = BASE_DIR / "~tmp_Datos_Modified.xlsx"

# Releer, forzar columnas y orden
df_out = pd.read_excel(final_excel)

# Garantiza que TODAS las columnas existan
for col in HEADERS:
    if col not in df_out.columns:
        df_out[col] = pd.NA

# Reordena exactamente como HEADERS
df_out = df_out[HEADERS]

# Escribe temporal
df_out.to_excel(tmp_excel, index=False)

# Reaplicar estilo vertical de headers
wb = load_workbook(tmp_excel)
ws = wb.active

# Congelar encabezado
ws.freeze_panes = "A2"

# Aplicar estilo a fila 1
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)  # bold=True si quieres negrita
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom", wrap_text=True)

wb.save(final_excel)

# Limpia temporal
try:
    tmp_excel.unlink()
except Exception as e:
    print("No se pudo borrar temporal:", e)

print("Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.")



Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.
